In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy as sp

import pandas as pd
import numpy as np


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nlplab/harry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# read training data
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [5]:
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer(text):
    return re.split('\s+', text.strip())

def tokenizer_stem(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in re.split('\s+', text.strip())]

def tokenizer_stem_nostop(text):
    stop = stopwords.words('english')
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

In [6]:
def fetch_datetime(soup):
    if soup.time.has_attr('datetime'):
        date = soup.time.attrs['datetime']
        day = ' '+ date[0:3]
    else:
        day  = ' fuckday'
    return day

def fetch_channel(soup):
    
    channel = soup.article['data-channel']
    return channel

def fetch_img_count(soup):

    c = 0
    find_all_images = soup.find_all('img')

    for i in find_all_images:
        c = c+1
    return c

def fetch_topics(soup):

    footer = soup.footer
    ta = footer.find_all('a')
    topic = []

    for t in ta:
        topic.append(t.get_text())
    topic_text = ' '.join(topic)

    return topic_text

def fetch_authors(soup):

    footer = soup.span
    if footer != None:
        ta = footer.findAll('a')
        authors = []
        for t in ta:
            authors.append(t.get_text())
        if len(authors) == 0:
            authors_text = 'NaN'
        else:
            authors_text = ''.join(authors)
    else:
        authors_text   = 'NaN'

    return authors_text

def fetch_titles(soup):
    
    footer = soup.h1
    if footer != None:
        titles = footer.get_text()
    else:
        titles = 'NaN'
    return titles

def fetch_social_media_count(soup):

    c = 0
    for frame in soup("iframe"):
        if frame.get('src').find("youtube") != None:
            c = c+1
        elif frame.get('src').find("instagram") != None:
            c = c+1
        elif frame.get('src').find("vine") != None:
            c = c+1

    return c



In [7]:
df_train_contents = df_train['Page content'].values.tolist()

In [8]:
def get_all_datas(texts):
    days = []
    channels = []
    img_counts = []
    topics = []
    authors = []
    titles = []
    social_media_counts = []
    contents = []
    
    for text in texts:
        soup = BeautifulSoup(text, "lxml")
        contents.append(soup.find('article').get_text())
        topics.append(fetch_topics(soup))
        channels.append(fetch_channel(soup))
        days.append(fetch_datetime(soup))
        authors.append(fetch_authors(soup))
        img_counts.append(fetch_img_count(soup))
        titles.append(fetch_titles(soup))
        social_media_counts.append(fetch_social_media_count(soup))
#         input()

    return days, channels, img_counts, topics, authors, titles, social_media_counts, contents

In [9]:
days, channels, img_counts, topics, authors, titles, social_media_counts, contents = get_all_datas(df_train_contents)

In [10]:
print(len(days))
print(len(channels))
print(len(img_counts))
print(len(topics))
print(len(authors))
print(len(titles))
print(len(social_media_counts))
print(len(contents))

27643
27643
27643
27643
27643
27643
27643
27643


In [11]:
# 2. n_tokens_title: Number of words in the title
# 3. n_tokens_content: Number of words in the content
# 4. n_unique_tokens: Rate of unique words in the content
# 5. n_non_stop_words: Rate of non-stop words in the content
# 6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
def process(text):
    
    # Remove HTML tags.
#     text = BeautifulSoup(text,'html.parser').get_text()
#     text = re.sub("[^a-zA-Z]", "", text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    
    token  = nltk.word_tokenize(text)
#     text = [porter.stem(w) for w in token if w not in stop]
    # Join the words back into one string separated by space, and return the result.
    return token

In [12]:
def get_some_n_features(titles, contents):
    n_tokens_titles = []
    n_tokens_contents = []
    n_unique_tokens = []
    n_non_stop_words = []
    n_non_stop_unique_tokens = []


    for title, content in zip(titles, contents):

        title_tokens = process(title)
        n_tokens_titles.append(len(title_tokens))

        content_tokens = process(content)
        n_tokens_contents.append(len(content_tokens))
        
        len_content_tokens = len(content_tokens)
        if len(content_tokens) == 0:
            print(content)
            len_content_tokens = 1
            
        
            

        set_content_token = set(content_tokens)
        unique_token_rate = len(set_content_token) / len_content_tokens
        n_unique_tokens.append(unique_token_rate)

        non_stop_words = [w for w in content_tokens if w not in stop]

        non_stop_word_rate = len(non_stop_words) / len_content_tokens
        n_non_stop_words.append(non_stop_word_rate)

        set_non_stop_words = set(non_stop_words)
        n_non_stop_unique_tokens_rate = len(set_non_stop_words) / len_content_tokens
        n_non_stop_unique_tokens.append(n_non_stop_unique_tokens_rate)


    return n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens
    
#     print(n_tokens_titles)
#     print(n_tokens_contents)
#     print(n_unique_tokens)
#     print(n_non_stop_words)
#     print(n_non_stop_unique_tokens)
    

In [13]:
n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens = get_some_n_features(titles, contents)




In [14]:
print(len(n_tokens_titles))
print(len(n_tokens_contents))
print(len(n_unique_tokens))
print(len(n_non_stop_words))
print(len(n_non_stop_unique_tokens))

27643
27643
27643
27643
27643


In [15]:
# re-consturct training data

# 2. n_tokens_title: Number of words in the title
# 3. n_tokens_content: Number of words in the content
# 4. n_unique_tokens: Rate of unique words in the content
# 5. n_non_stop_words: Rate of non-stop words in the content
# 6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content
df_train_feature = pd.DataFrame({'Id':df_train.Id[:],
                           'Popularity':df_train.Popularity[:],
                           'topic':topics,
                           'channel':channels,
                           'weekday':days,
                           'author':authors,
                           'img count':img_counts,
                           'title':titles,
                            'content':contents,
                           'media count': social_media_counts,
                           'n_tokens_title' : n_tokens_titles,
                           'n_tokens_content': n_tokens_contents,
                           'n_unique_tokens' : n_unique_tokens,
                           'n_non_stop_words': n_non_stop_words,
                            'n_non_stop_unique_tokens': n_non_stop_unique_tokens})

In [16]:
df_train_feature.to_csv('./train_feature.csv',index=False,header=True)

In [ ]:
df_test_contents = df_test['Page content'].values.tolist()
print(len(df_test_contents))
days, channels, img_counts, topics, authors, titles, social_media_counts, contents = get_all_datas(df_test_contents)
print(len(days))
print(len(channels))
print(len(img_counts))
print(len(topics))
print(len(authors))
print(len(titles))
print(len(social_media_counts))
print(len(contents))


11847


In [ ]:
n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens = get_some_n_features(titles, contents)


In [ ]:
print(len(n_tokens_titles))
print(len(n_tokens_contents))
print(len(n_unique_tokens))
print(len(n_non_stop_words))
print(len(n_non_stop_unique_tokens))

In [ ]:
# re-consturct training data
df_test_feature = pd.DataFrame({'Id':df_test.Id[:],
                           'topic':topics,
                           'channel':channels,
                           'weekday':days,
                           'author':authors,
                           'img count':img_counts,
                           'title':titles,
                            'content':contents,
                           'media count': social_media_counts,
                           'n_tokens_title' : n_tokens_titles,
                           'n_tokens_content': n_tokens_contents,
                           'n_unique_tokens' : n_unique_tokens,
                           'n_non_stop_words': n_non_stop_words,
                            'n_non_stop_unique_tokens': n_non_stop_unique_tokens})

In [ ]:
df_test_feature.to_csv('./test_feature.csv',index=False,header=True)